In [1]:
import spacy
import pdfminer
import re
import os 
import pandas as pd
import pdf2txt

In [2]:
def convert_pdf(file):
    output_filename = os.path.basename(os.path.splitext(file)[0]) + ".txt"
    output_filepath = os.path.join("output/txt/", output_filename)
    pdf2txt.main(args=[file, "--outfile", output_filepath])
    print(output_filepath + " saved succesfully!!!")
    return open(output_filepath, encoding='utf8').read()

In [6]:
# Loading the language model

nlp = spacy.load("en_core_web_sm")

In [7]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []}
names = []
phones = []
emails = []
skills = []

In [8]:
# phone_num = https://stackoverflow.com/a/3868861
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile(
        "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    )
    doc = nlp(text)    
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][3]
    print(name)
    email = [word for word in doc if word.like_email == True]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset,text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [9]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading....' + file)
        txt = convert_pdf(os.path.join('resumes/', file))
        parse_content(txt)

Reading....Aarón Curriculum.pdf
output/txt/Aarón Curriculum.txt saved succesfully!!!
Aarón Juárez
[aaronga@comunidad.unam.mx]
Extraction completed successfully!!!


In [7]:
names 

['Aarón Juárez']

In [8]:
phones

["['5568533536']"]

In [9]:
emails

[[aaronga@comunidad.unam.mx]]

In [10]:
skills

["{'sql', 'python'}"]

In [11]:
result_dict['name'] = names
result_dict['phone']= phones
result_dict['email']= emails
result_dict['skills']=skills
print(result_dict)


{'name': ['Aarón Juárez'], 'phone': ["['5568533536']"], 'email': [[aaronga@comunidad.unam.mx]], 'skills': ["{'sql', 'python'}"]}


In [12]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Aarón Juárez,['5568533536'],[aaronga@comunidad.unam.mx],"{'sql', 'python'}"


In [14]:
result_df.to_csv('output/csv/parsed_resume.csv')
